In [ ]:
!pip install datasets deepeval

In [2]:
import os
import json
import random

import numpy as np
from openai import OpenAI
from datasets import load_dataset
from scipy.stats import ttest_rel, ttest_1samp

from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase
from deepeval.test_case import LLMTestCaseParams

In [3]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [4]:
client = OpenAI()

In [5]:
style_matching_metric_geval = GEval(
    name="Style Matching",
    criteria="Определи, насколько последняя фраза (actual output) делает анекдот смешным, логичным и остроумным:",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT]
)

semantic_coherence_metric = GEval(
    name="Semantic Coherence",
    criteria="Оцени, насколько последняя фраза (actual output) завершает анекдот, сохраняя смысловую и контекстуальную связь с началом (input).",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT]
)

In [6]:
system_message1 = {"role": "system",
                   "content": "Ты чат бот, который генерирует анекдоты. "
                              "Продолжи диалог одной репликой так, чтоб получился анекдот. "
                              "Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? "
                              "Expected output: - Мои!"}
system_message2 = {"role": "system",
                   "content": "Ты чат бот, который генерирует диалоги. Продолжи диалог"}

In [8]:
def prepare_openai_jokes_dataset(num_samples: int):
    ds = load_dataset("inkoziev/jokes_dialogues")
    ds_df = ds['train'].to_pandas()

    # print(ds_df)
    unique_chat_ids = ds_df['src_hash'].unique()
    # unique_chat_ids = set(ds['train']['src_hash'])
    sampled_chat_ids = np.random.choice(list(unique_chat_ids), num_samples, replace=False)

    openai_chat_examples = []
    for chat_id in sampled_chat_ids:
        reply_nums = ds_df[ds_df['src_hash'] == chat_id]['reply_num'].values
        reply_nums = np.sort(reply_nums)
        messages = [system_message1]
        if reply_nums[0] != 1:
            continue
        message1 = ds_df[(ds_df['src_hash'] == chat_id) & (ds_df['reply_num'] == 1)]['context'].values[0]
        messages.append({"role": "assistant" if 0 % 2 == reply_nums[-1] % 2 else "user",
                         "content": message1})

        for reply_num in reply_nums:
            message = ds_df[(ds_df['src_hash'] == chat_id) & (ds_df['reply_num'] == reply_num)]['utterance'].values[0]
            message_metadata = {"role": "assistant" if reply_num % 2 == reply_nums[-1] % 2 else "user",
                                "content": message}
            if message_metadata['role'] == 'assistant':
                if reply_num == reply_nums[-1]:
                    message_metadata['weight'] = 1
                else:
                    message_metadata['weight'] = 0
            messages.append(message_metadata)

        openai_chat_example = {"messages": messages}
        openai_chat_examples.append(openai_chat_example)
    return openai_chat_examples

In [9]:
def get_model_predictions(gt_examples, system_message, used_model='gpt-4o-mini'):
    predicted_examples = []
    for i_example in range(len(gt_examples)):
        completion = client.chat.completions.create(
            model=used_model,
            messages=[system_message] + gt_examples[i_example]['messages'][1:-1],
        )
        # predicted_examples.append(completion.choices[0].message.content)
        predicted_example = [system_message] + gt_examples[i_example]['messages'][1:-1]
        predicted_example.append({"role": "assistant", "content": completion.choices[0].message.content})
        predicted_example = {"messages": predicted_example}
        predicted_examples.append(predicted_example)

    return predicted_examples

In [10]:
def evaluate_metrics(gt_examples, predicted_examples1, predicted_examples2):
    scores1 = []
    scores2 = []
    sc_fit_scores1 = []
    sc_fit_scores2 = []

    for i_example in range(len(gt_examples)):
        test_case1 = LLMTestCase(
            input="\n".join([msg['content'] for msg in predicted_examples1[i_example]['messages'][1:-1]]),
            actual_output=predicted_examples1[i_example]['messages'][-1]['content'],
            expected_output=gt_examples[i_example]['messages'][-1]['content'],
        )
        test_case2 = LLMTestCase(
            input="\n".join([msg['content'] for msg in predicted_examples2[i_example]['messages'][1:-1]]),
            actual_output=predicted_examples2[i_example]['messages'][-1]['content'],
            expected_output=gt_examples[i_example]['messages'][-1]['content'],
        )

        # Style Matching
        style_matching_metric_geval.measure(test_case1)
        scores1.append(style_matching_metric_geval.score)
        style_matching_metric_geval.measure(test_case2)
        scores2.append(style_matching_metric_geval.score)

        # Completion Humor Fit
        semantic_coherence_metric.measure(test_case1)
        sc_fit_scores1.append(semantic_coherence_metric.score)
        semantic_coherence_metric.measure(test_case2)
        sc_fit_scores2.append(semantic_coherence_metric.score)

    return gt_examples, predicted_examples1, predicted_examples2, scores1, scores2, sc_fit_scores1, sc_fit_scores2

In [12]:
def main(num_samples: int):
    gt_examples = prepare_openai_jokes_dataset(num_samples)
    print("\n============== GT EXAMPLES ===============")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print("-----------")
    predicted_examples1 = get_model_predictions(gt_examples, system_message1)
    predicted_examples2 = get_model_predictions(gt_examples, system_message2)
    print("\n=============== GT and Predicted EXAMPLES ======================")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print(predicted_examples1[i_example])
        print(predicted_examples2[i_example])
        print("-----------")
    gt_examples, predicted_examples1, predicted_examples2, \
        scores1, scores2, sc_fit_scores1, sc_fit_scores2 = evaluate_metrics(
            gt_examples,
            predicted_examples1,
            predicted_examples2
        )

    print("\n=============== GT and Predicted EXAMPLES with METRICS ======================")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print(predicted_examples1[i_example])
        print(predicted_examples2[i_example])
        print("-----------")

    print("\n=============== Scores ======================")
    ttest_res1 = ttest_1samp(scores1, 0.5)
    print(f"\nSCORE 1: {np.mean(scores1):.3f}")
    print(", ".join([f"{x:.3f}" for x in scores1]))
    print(f"\nTEST 1: {ttest_res1}")
    ttest_res2 = ttest_1samp(scores2, 0.5)
    print(f"\nSCORE 2: {np.mean(scores2):.3f}")
    print(", ".join([f"{x:.3f}" for x in scores2]))
    print(f"\nTEST 2: {ttest_res2}")

    rel_ttest_res = ttest_rel(scores1, scores2)

    print("\n=============== Rel Scores ======================")
    print(f"\nTEST: {rel_ttest_res}")

    print("\n=============== My Scores ======================")
    ttest_res1 = ttest_1samp(sc_fit_scores1, 0.5)
    print(f"\nSCORE 1: {np.mean(sc_fit_scores1):.3f}")
    print(", ".join([f"{x:.3f}" for x in sc_fit_scores1]))
    print(f"\nTEST 1: {ttest_res1}")
    ttest_res2 = ttest_1samp(sc_fit_scores2, 0.5)
    print(f"\nSCORE 2: {np.mean(sc_fit_scores2):.3f}")
    print(", ".join([f"{x:.3f}" for x in sc_fit_scores2]))
    print(f"\nTEST 2: {ttest_res2}")

    my_rel_ttest_res = ttest_rel(sc_fit_scores1, sc_fit_scores2)

    print("\n=============== Rel My Scores ======================")
    print(f"\nTEST: {my_rel_ttest_res}")

In [13]:
main(num_samples=30)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

jokes_dialog_dataset.json:   0%|          | 0.00/61.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/114579 [00:00<?, ? examples/s]


============== GT EXAMPLES ===============
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': 'Едет грузовик, его останавливает ГИБДДэшник:\n- Слышь, мужик, ты что - [бах]нутый? Я тебя третий раз останавливаю и третий раз говорю: у тебя из кузова сыпется что-то.'}, {'role': 'assistant', 'content': 'Слышь, ты меня хоть восьмой раз останови, я тебе скажу: Зима! Гололед!', 'weight': 1}]}
-----------
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': 'Один гаец побывал в Штатах на стажировке. Вернулс

...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...


=============== GT and Predicted EXAMPLES with METRICS ======================
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': 'Едет грузовик, его останавливает ГИБДДэшник:\n- Слышь, мужик, ты что - [бах]нутый? Я тебя третий раз останавливаю и третий раз говорю: у тебя из кузова сыпется что-то.'}, {'role': 'assistant', 'content': 'Слышь, ты меня хоть восьмой раз останови, я тебе скажу: Зима! Гололед!', 'weight': 1}]}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': 'Едет грузовик, его останав

**Вывод:**


---


Результаты стандартной метрики:

SCORE 1: Среднее значение 0.443.

SCORE 2: Среднее значение 0.316.

Статистический тест для SCORE 1 показал (p = 0.0176) → различия являются статистически значимыми (p > 0.05) по сравнению со значением 0.5.

Для SCORE 2 различия оказались статистически значимыми (p ~= 0.0001) → вывод аналогичный p значению у SCORE 1.

Таким образом данная оценка показывает, что модель лучше работает для написания анекдотов, чем для продолжения диалогов (хотя и как писатель анекдотов она имеет не высокую оценку)


---


Результаты кастомной метрики:

SCORE 1: Среднее значение 0.815.

SCORE 2: Среднее значение 0.516.

Статистический тест для SCORE 1 выявил статистически значимые различия (p = 0.0001) → результат не случайно хороший.

Для SCORE 2 различия не являются статистически значимыми (p = 0.6951) → результат статистически не отличим от 0.5.

Результаты данной метрики показывают, что модель хорошо логически связывает начала и концы анекдотов, когда она выступает в роли писателя анекдотов.

In [15]:
jokes_data = load_dataset("inkoziev/jokes_dialogues")

In [16]:
# all_indices = list(range(len(jokes_data['train'])))

# sampled_indices = random.sample(all_indices, 50)

sampled_indices = [
    78841, 37873, 48611, 19285, 59437, 30713, 109711, 29000,
    51294, 54427, 46481, 79965, 43178, 104276, 83993, 42483,
    85775, 56299, 36301, 29918, 76590, 31351, 11555, 73633,
    63912, 97828, 16790, 104108, 50671, 49612, 107142, 53392,
    56905, 23546, 72411, 64468, 22670, 29961, 60337, 94127, 14883,
    31735, 102630, 24885, 97753, 19525, 1274, 4390, 100677, 54711]

jokes_subset = jokes_data['train'].select(sampled_indices)

In [17]:
jokes_list = jokes_subset.to_list()
print(len(jokes_list))

50


In [18]:
def reformat_dictionary(joke):
    system_message = {"role": "system", "content": "Ты чат бот, который генерирует анекдоты."}
    user_message = {"role": "user", "content": joke['context']}
    assistant_message = {"role": "assistant", "content": joke['utterance']}

    reformatted_dictionary = {
        "messages": [system_message, user_message, assistant_message]
    }
    return reformatted_dictionary

In [19]:
reformatted_jokes_list = [reformat_dictionary(joke) for joke in jokes_list]

In [20]:
with open('jokes_subset.jsonl', 'w', encoding='utf-8') as f:
    for joke in reformatted_jokes_list:
        f.write(json.dumps(joke, ensure_ascii=False) + '\n')

In [21]:
!curl https://api.openai.com/v1/files \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -F purpose="fine-tune" \
  -F file="@/content/jokes_subset.jsonl"

{
  "object": "file",
  "id": "file-5GN68pKine4aHM2cykeKXd",
  "purpose": "fine-tune",
  "filename": "jokes_subset.jsonl",
  "bytes": 26842,
  "created_at": 1734809461,
  "status": "processed",
  "status_details": null
}


In [22]:
client.fine_tuning.jobs.create(
    training_file="file-NED1or8goUcAR7QX6rWA4o",
    model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-8egFWxIbjN7jf3rp2KLq61t1', created_at=1734809464, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yyyPGCw9VCJjgaycEyhn8xFf', result_files=[], seed=784755585, status='validating_files', trained_tokens=None, training_file='file-NED1or8goUcAR7QX6rWA4o', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)

In [23]:
fine_tuning_job = client.fine_tuning.jobs.retrieve("ftjob-ic5HOqKqDT0V92CeKe3B5dD4")
print(fine_tuning_job)

FineTuningJob(id='ftjob-ic5HOqKqDT0V92CeKe3B5dD4', created_at=1734727157, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::Age9sPkO', finished_at=1734727506, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yyyPGCw9VCJjgaycEyhn8xFf', result_files=['file-L7ijM4FyiGDmXX6WHbYTed'], seed=1556080021, status='succeeded', trained_tokens=12444, training_file='file-NED1or8goUcAR7QX6rWA4o', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3)), type='supervised'), user_provided_suffix=None)


In [24]:
completion = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::Age9sPkO",
    messages=[
        {"role": "system", "content": "Ты чат бот, который придумывает анекдоты из предложенной фразы."},
        {"role": "user", "content": "Слушал сегодня послание Федеральному Собранию?"}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Нет, мне песенку вежливого детёнышка послушать надо.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [26]:
def main_2(num_samples: int):
    gt_examples = prepare_openai_jokes_dataset(num_samples)
    print("\n============== GT EXAMPLES ===============")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print("-----------")
    predicted_examples1 = get_model_predictions(gt_examples, system_message1, used_model="ft:gpt-4o-mini-2024-07-18:personal::Age9sPkO")
    predicted_examples2 = get_model_predictions(gt_examples, system_message2, used_model="ft:gpt-4o-mini-2024-07-18:personal::Age9sPkO")
    print("\n=============== GT and Predicted EXAMPLES ======================")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print(predicted_examples1[i_example])
        print(predicted_examples2[i_example])
        print("-----------")
    gt_examples, predicted_examples1, predicted_examples2, \
        scores1, scores2, sc_fit_scores1, sc_fit_scores2 = evaluate_metrics(
            gt_examples,
            predicted_examples1,
            predicted_examples2
        )

    print("\n=============== GT and Predicted EXAMPLES with METRICS ======================")
    for i_example in range(len(gt_examples)):
        print(gt_examples[i_example])
        print(predicted_examples1[i_example])
        print(predicted_examples2[i_example])
        print("-----------")

    print("\n=============== Scores ======================")
    ttest_res1 = ttest_1samp(scores1, 0.5)
    print(f"\nSCORE 1: {np.mean(scores1):.3f}")
    print(", ".join([f"{x:.3f}" for x in scores1]))
    print(f"\nTEST 1: {ttest_res1}")
    ttest_res2 = ttest_1samp(scores2, 0.5)
    print(f"\nSCORE 2: {np.mean(scores2):.3f}")
    print(", ".join([f"{x:.3f}" for x in scores2]))
    print(f"\nTEST 2: {ttest_res2}")

    rel_ttest_res = ttest_rel(scores1, scores2)

    print("\n=============== Rel Scores ======================")
    print(f"\nTEST: {rel_ttest_res}")

    print("\n=============== My Scores ======================")
    ttest_res1 = ttest_1samp(sc_fit_scores1, 0.5)
    print(f"\nSCORE 1: {np.mean(sc_fit_scores1):.3f}")
    print(", ".join([f"{x:.3f}" for x in sc_fit_scores1]))
    print(f"\nTEST 1: {ttest_res1}")
    ttest_res2 = ttest_1samp(sc_fit_scores2, 0.5)
    print(f"\nSCORE 2: {np.mean(sc_fit_scores2):.3f}")
    print(", ".join([f"{x:.3f}" for x in sc_fit_scores2]))
    print(f"\nTEST 2: {ttest_res2}")

    my_rel_ttest_res = ttest_rel(sc_fit_scores1, sc_fit_scores2)

    print("\n=============== Rel My Scores ======================")
    print(f"\nTEST: {my_rel_ttest_res}")

In [27]:
main_2(num_samples=30)


============== GT EXAMPLES ===============
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'assistant', 'content': 'Парень поступает в медицинский институт, на отделение "гинекология"\n. Его спрашивают:'}, {'role': 'user', 'content': 'А что это: - ваш прадед гинекологом был, дед, отец - это династия?'}, {'role': 'assistant', 'content': 'Нет, насмотреться не можем.', 'weight': 1}]}
-----------
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'assistant', 'content': 'Был свидетелем. Банкет инвалидов-КВНщиков. Один без кистей рук, 

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...


=============== GT and Predicted EXAMPLES ======================
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'assistant', 'content': 'Парень поступает в медицинский институт, на отделение "гинекология"\n. Его спрашивают:'}, {'role': 'user', 'content': 'А что это: - ваш прадед гинекологом был, дед, отец - это династия?'}, {'role': 'assistant', 'content': 'Нет, насмотреться не можем.', 'weight': 1}]}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'assistant', 'content': 'Парень поступает в медицинский институт, на отделение 

Output()

...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...


=============== GT and Predicted EXAMPLES with METRICS ======================
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'assistant', 'content': 'Парень поступает в медицинский институт, на отделение "гинекология"\n. Его спрашивают:'}, {'role': 'user', 'content': 'А что это: - ваш прадед гинекологом был, дед, отец - это династия?'}, {'role': 'assistant', 'content': 'Нет, насмотреться не можем.', 'weight': 1}]}
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'assistant', 'content': 'Парень поступает в медицинский институт, 

**Вывод:**


---


Результаты стандартной метрики:

SCORE 1: Среднее значение 0.309. (В базовой модели значение 0.443)

SCORE 2: Среднее значение 0.329. (В базовой модели значение 0.316)

Для SCORE 1 статистический тест показал значимые различия (p ~= 0.0001) по сравнению с 0.5. (В базовой модели значение 0.0176)

Для SCORE 2 статистический тест не выявил значимые различия (p ~= 0.0001) с 0.5. (В базовой модели значение ~ 0.0001)

Т.е. затюниная модель стала чуть лучше работать как продолжатель диалогов, но стала хуже работать как писатель анекдотов.


---


Результаты кастомной метрики:

SCORE 1: Среднее значение 0.489. (В базовой модели значение 0.815)

SCORE 2: Среднее значение 0.528. (В базовой модели значение 0.516)

Для SCORE 1 статистический тест не выявил значимых различий (p = 0.8344) по сравнению с 0.5. (В базовой модели значение ~ 0.0001)

Для SCORE 2 также не было выявлено статистически значимых различий (p = 0.6061) (В базовой модели значение 0.6951)

Т.е. затюниная модель стала хуже работать как писатель анекдотов для кастомной метрики


---


Ухудшение работы модели может быть связано с недостаточным объемом датасета и с плохо сформированными анекдотами в нём.